In [ ]:
import numpy as np
from lenspack.utils import bin2d
from lenspack.image.inversion import ks93
from lenspack.geometry.projections.gnom import radec2xy
from scipy import ndimage as ndi
import matplotlib.pyplot as plt

In [ ]:
#this notebook creates a data file like peaks_mean_global.npy
# tasks in this notebook
#1. load catalog
#2. calibrate shears
#3. switch to pixels
#4. bin galaxy shears
#5. make map
#6. cut out patches
#7. gaussian filtering
#8. divide by noise to get SNR maps
#9. measure peaks
#10. combine information from patches 

In [ ]:
#1. load catalog
dd = np.load('final_cat.npy', mmap_mode=None)
dd.dtype

In [ ]:
#2. shear calibration
# define shear response matrix (without R_s)
DGAMMA=0.02
R11=(dd['NGMIX_ELL_1P'][:,0]-dd['NGMIX_ELL_1M'][:,0])/DGAMMA
R22=(dd['NGMIX_ELL_2P'][:,1]-dd['NGMIX_ELL_2M'][:,1])/DGAMMA
R12=(dd['NGMIX_ELL_2P'][:,0]-dd['NGMIX_ELL_2M'][:,0])/DGAMMA
R21=(dd['NGMIX_ELL_1P'][:,1]-dd['NGMIX_ELL_1M'][:,1])/DGAMMA


In [ ]:
# define cuts
# shear independent- note: there are problems here, the mag cuts and star cuts are DEF shear dependent, and the star galaxy separation method has been updated
shear_independent_cut = \
    (dd['SPREAD_MODEL']+2*dd['SPREADERR_MODEL'] > 0.0035) \
    & (dd['SPREAD_MODEL'] > 0.) \
    & (dd['SPREAD_MODEL'] < 0.03) \
    & (dd['FLAGS'] == 0) \
    & (dd['IMAFLAGS_ISO'] == 0) \
    & (dd['NGMIX_MCAL_FLAGS'] == 0) \
    & (dd['NGMIX_ELL_PSFo_NOSHEAR'][:,0] != -10) \
    & (dd['NGMIX_MOM_FAIL'] == 0) \
    & (dd['N_EPOCH'] > 0) \
    & (dd['NGMIX_N_EPOCH'] > 0)

# shear dependent
cutcat=[]
shears=['NOSHEAR','1P','1M','2P','2M']
for shear in shears:
    # define cut quantities
    snr_ngmix = dd['NGMIX_FLUX_'+shear]/dd['NGMIX_FLUX_ERR_'+shear]
    rel_size = dd['NGMIX_T_'+shear]/dd['NGMIX_Tpsf_'+shear]
    # cuts
    snr_min= snr_ngmix > 10
    snr_max= snr_ngmix < 500
    rel_size_min = rel_size > np.sqrt(.5) 
    
    good=(shear_independent_cut)&(snr_min)&(snr_max)&(rel_size_min)
    
    cutcat.append(good)    


In [ ]:
# calculate selection response
R11_s=((np.average(dd['NGMIX_ELL_NOSHEAR'][np.where(cutcat[1]),0])-np.average(dd['NGMIX_ELL_NOSHEAR'][np.where(cutcat[2]),0]))/DGAMMA)
print(R11_s)
R22_s=((np.average(dd['NGMIX_ELL_NOSHEAR'][np.where(cutcat[3]),1])-np.average(dd['NGMIX_ELL_NOSHEAR'][np.where(cutcat[4]),1]))/DGAMMA)
print(R22_s)
R12_s=((np.average(dd['NGMIX_ELL_NOSHEAR'][np.where(cutcat[3]),0])) - (np.average(dd['NGMIX_ELL_NOSHEAR'][np.where(cutcat[4]),0])))/(DGAMMA)
print(R12_s)
R21_s=((np.average(dd['NGMIX_ELL_NOSHEAR'][np.where(cutcat[1]),1])) - (np.average(dd['NGMIX_ELL_NOSHEAR'][np.where(cutcat[2]),1])))/DGAMMA
print(R21_s)


In [ ]:
# get total shear response and invert it (we can do this in a less dumb way later)
R11_tot=np.mean(R11)+R11_s
R22_tot=np.mean(R22)+R22_s
R21_tot=np.mean(R21)+R21_s
R12_tot=np.mean(R12)+R12_s
print(R11_tot)
print(R22_tot)
DetR=(R11_tot*R22_tot-R21_tot*R12_tot)
R11_inv=1/DetR*(R22_tot)
R22_inv=1/DetR*(R11_tot)
R12_inv=1/DetR*(-R12_tot)
R21_inv=1/DetR*(-R21_tot)

In [ ]:
# calibrate shear
g1_corr=R11_inv*dd['NGMIX_ELL_NOSHEAR'][:,0][cutcat[0]]
g2_corr=R22_inv*dd['NGMIX_ELL_NOSHEAR'][:,1][cutcat[0]]

In [ ]:
plt.hist(g1_corr)
plt.hist(dd['NGMIX_ELL_NOSHEAR'][:,0][cutcat[0]])

In [ ]:
#3. Compute number of pixels, Switch from RA, DEC to pixels
# first, mask galaxies
ra_ngmix = dd['XWIN_WORLD'][np.where(cutcat[0])]
dec_ngmix = dd['YWIN_WORLD'][np.where(cutcat[0])]

size_x_deg = 44
size_y_deg = 16
pixel_size_emap_amin=0.4

Nx = int(size_x_deg / pixel_size_emap_amin * 60)
Ny = int(size_y_deg / pixel_size_emap_amin * 60)

print(Nx,Ny)
import lenspack
# Project (ra,dec) -> (x,y)
x, y = radec2xy(np.mean(ra_ngmix), np.mean(dec_ngmix), ra_ngmix, dec_ngmix)

In [ ]:
#4. bin calibrated shears in 2D
# v are the values, w is another option that tells you the weights
# do we weight these? not according to what martin has
# should we add the extent extent=(min_x[sh], max_x[sh], min_y[sh], max_y[sh])
# used for extent? check if default is okay
min_x = np.min(x)
max_x = np.max(x)
min_y = np.min(y)
max_y = np.max(y)

size_x = max_x - min_x
size_y = max_y - min_y

e1map, e2map = bin2d(x, y, npix=(Nx,Ny), v=(g1_corr,g2_corr))
emap = np.array([e1map,e2map])




In [ ]:
#5. make kappa map- note, the minus sign has to be here for our data conventions
kappaE_cal, kappaB_cal = ks93(e1map,-e2map)

In [ ]:
#6. make cutouts here, size 512, 512

In [ ]:
# run all remaining steps per cutout

In [ ]:
#7. smooth kappa map with gaussian filter
precision_Peaks = 2/pixel_size_emap_amin

kappa_noisy_smoothed_cal = ndi.gaussian_filter(kappaE_cal, precision_Peaks) # Global
kappa_noisy_smoothedB = ndi.gaussian_filter(kappaB_cal, precision_Peaks) #ei/Rii

In [ ]:
#8. convert kappa map to SNR map

# define noise properties 
#galaxy number density in gal/arcmin^2
n_gal=7
#pixel size in arcmin
pix_arcmin=0.4
#compute shape noise for CFIS
shape_noise=0.44
sigma_noise_CFIS=shape_noise/(np.sqrt(2*n_gal*pix_arcmin**2))
# generate noise map
noise_CFIS_z05=sigma_noise_CFIS*np.random.randn(512, 512)

# compute snr map
snr_cal = kappa_noisy_smoothed_cal/np.std(ndi.gaussian_filter(noise_CFIS_z05, precision_Peaks))
snrB = kappa_noisy_smoothedB/np.std(ndi.gaussian_filter(noise_CFIS_z05, precision_Peaks))


In [ ]:

#8. measuring peaks
# we want peak counts computed on SNR maps, but we can also compute them directly on kappa maps

# run peak counts on kappa maps
kappa_th = np.linspace(-0.06, 0.06, 31)
kappa_th_center = 0.5*(kappa_th[:-1]+kappa_th[1:])

peak_counts_cal, bins_cal = peaks.peaks_histogram(kappa_noisy_smoothed_cal,kappa_th)

# compute peak counts on snr maps- this will look like input to likelihood code
kappa_snr = np.linspace(-2,6,31)
kappa_th_center_snr = 0.5*(kappa_snr[:-1]+kappa_snr[1:])

peak_counts_cal_snr, bins_cal = peaks.peaks_histogram(snr_cal,kappa_snr)

In [ ]:
#plot single-Gaussian peak counts- you can compare this for different mass mapping methods.  we can try per patch and also comparing the mean across 13 patches
plt.figure(figsize=(9,6))
plt.plot(kappa_th_center,peak_counts_cal,'b-',label='Global cal')
plt.legend()
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.xlabel('kappa smooth', fontsize=18)
plt.ylabel('peak counts', fontsize=18)
plt.show()

In [ ]:
#mean of the peak counts from the 13 patches